# Финальный проект. Декомпозиция

## Описание проекта

Датасет содержит данные о событиях, совершенных в мобильном приложении "Ненужные вещи". В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений.
В датасете содержатся данные пользователей, впервые совершивших действия в
приложении после 7 октября 2019 года.

Колонки в /datasets/mobile_sources.csv :
- userId — идентификатор пользователя,
- source — источник, с которого пользователь установил приложение.
<br>

Колонки в /datasets/mobile_dataset.csv :
- event.time — время совершения,
- user.id — идентификатор пользователя,
- event.name — действие пользователя.
<br>

Виды действий:
- advert_open — открыл карточки объявления,
- photos_show — просмотрел фотографий в объявлении,
- tips_show — увидел рекомендованные объявления,
- tips_click — кликнул по рекомендованному объявлению,
- contacts_show и show_contacts — посмотрел номер телефона,
- contacts_call — позвонил по номеру из объявления,
- map — открыл карту объявлений,
- search_1 — search_7 — разные действия, связанные с поиском по сайту,
- favorites_add — добавил объявление в избранное.

## Задачи проекта

1. Проанализировать связь целевого события — просмотра контактов — и других действий пользователей.
2. Оценить, какие действия чаще совершают те пользователи, которыепросматривают контакты.

## Цели

Анализ поведения пользователей приложения "Ненужные вещи" для увеличения конверсии в целевое действие, что в свою очередь увеличит вовлеченость

## Структура проекта




- __Знакомство с данными__
- __Предобработка данных__ 
    - Проверка дубликатов 
    - Проверка пропусков 
    - Ошибки в данных
    - Названия столбцов
- __Иследовательский анализ данных__
    - Время проведеное в приложении
    - Retention Rate
- __Основные вопросы иследования__
    - В разрезе сессий отобрать сценарии, которые приводят к просмотру контактов
    - Строим воронки по основным сценариям в разрезе уникальных пользователей
    - Различия длитильности сессии с просмотром номера и без
    - Рассчитываем относительную частоту событий для тех кто смотрел контакты и тех кто не смотрел
    


- __Проверка гипотиз__
    - Одни пользователи совершают действия tips_show и tips_click, другие – только tips_show. Проверить гипотезу, что конверсия в просмотр контактов различается у этих двух групп.
    - Различается ли конверсия в целевое действие пользователей, которые добавляют объявление в избраное, от тех кто не добавляет.